In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
proj_key = os.getenv('OPENAI_PROJ_KEY')

client = OpenAI(project = proj_key)

data = pd.read_csv('~/Documents/SOCY3001/SOCY3001/rawdata.csv')
data = data
data

output = pd.DataFrame(columns=['Location', 'Data', 'Goal', 'Coordinates'])
output = []

In [196]:
for i in range(len(data)):
    prompt = f"Given the following research description, provide the following information:\n\
    1) The location where the research was conducted, if specified. If it is completely impossible to say where the research was conducted, respond with 'Not specified'. Be as specific as possible.\n\
    2) The type of data used in the research (e.g. satellite imagery, remote sensing data). If it is completely impossible to even make a guess, respond with 'Not specified'\n\
    3) The goal of the research (e.g. predicting forest fires, classifying tree species). This should specifically be the GOAL or OBJECTIVE of the research, not a general keyword or description of the research conducted.\n\
    4) Your estimation of the coordinates where the research was conducted, separated by ', '. Only provide one set of coordinates. If your answer to (1) was that the location is not specified, respond with 'Not specified' here as well. If the research is being conducted globally or not in a specific place, respond with 'Not specified'. If only a country/general region is specified, make your best guess at the approximate coordinates, or pick a random set of coordinates within that region.\n\
    The coordinates must be formatted like 12.345678, 12.345678. Do not use brackets, letters, extra spaces or punctuation, or anything else. Just the coordinates separated by ', '. You must format the coordinates perfectly - this is absolutely critical, as otherwise everything will break. DO NOT FORMAT THE COORDINATES INCORRECTLY. \n\
    If the research is not related in any way to using artificial intelligence/machine learning/deep learning to study forests/trees/woodlands, return N/A for all questions. \n\\n\
    Give your response as exactly FOUR values - your answers to each of the four questions - separated by semicolons. Each output MUST have exactly four responses separated by exactly three semicolons - if you don't use exactly 3 semicolons, everything will break, so getting the formatting correct is of critical importance. An example of the output structure is 'China;Satellite images;Forest fire detection;41.40338, 2.17403' (do not use quotes).\n\
    Where possible, the data used and the goal of the research should be general and capable of being aggregated with other, similar research (e.g. 'Remote sensing data' or 'Tree species classification').\n\n\
    Research description:\n{data.iloc[i,0]}\n\
    {data.iloc[i,1]}\n\n\
    {data.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output.append(response.choices[0].message.content.split(';')) 

In [207]:
for inner_list in output:
    if inner_list == ['N/A']:
        inner_list[:] = ["N/A", "N/A", "N/A", "N/A"]
    if inner_list == ['Argentina', 'Not specified', 'Biodiversity conservation', 'Gran Chaco region', ' -24.000000, -60.000000']:
        inner_list[:] = ['Argentina', 'Not specified', 'Biodiversity conservation', ' -24.000000, -60.000000']
    if inner_list == ['Bangladesh', 'Landsat satellite images', 'Modeling forest cover dynamics', '23.685', '90.3563']:
        inner_list[:] = ['Bangladesh', 'Landsat satellite images', 'Modeling forest cover dynamics', '23.685, 90.3563']
    if inner_list == ['Venezuela', 'Not specified', 'Plant phenology disruption', '8. Venezuela', ' -66.927438, 6.423750']:
        inner_list[:] = ['Venezuela', 'Not specified', 'Plant phenology disruption', '-66.927438, 6.423750']
    if inner_list == ['Roraima State, Brazil', 'Pollen, spores, microcharcoal, loss on ignition data', 'Analyzing past forest-savanna dynamics', 'Roraima State, Brazil', ' 4.67143, -61.12855']:
        inner_list[:] = ['Roraima State, Brazil', 'Pollen, spores, microcharcoal, loss on ignition data', 'Analyzing past forest-savanna dynamics','4.67143, -61.12855']
    if inner_list == ['Bangladesh', 'Not specified', 'Soil carbon and nutrient cycling', '23.685', '90.3563']:
        inner_list[:] = ['Bangladesh', 'Not specified', 'Soil carbon and nutrient cycling', '23.685, 90.3563']
    if inner_list == ['Belem', 'Not specified', 'Climate influence on litter in a forest', '1. preparing data for machine learning research based on forest dynamics', ' -1.45542, -48.50309']:
        inner_list[:] = ['Belem', 'Not specified', 'Climate influence on litter in a forest', '-1.45542, -48.50309']
    if inner_list == ['Bangladesh', 'Not specified', 'Forest regeneration evaluation', '23.685', '90.3563']:
        inner_list[:] = ['Bangladesh', 'Not specified', 'Forest regeneration evaluation', '23.685, 90.3563']
    
    
for inner_list in output:
    if len(inner_list) != 4:
        print(inner_list)

In [208]:
output_clean = output
output_df = pd.DataFrame(output_clean, columns=['Location', 'Data', 'Goal', 'Coordinates'])

file_path = '~/Documents/SOCY3001/SOCY3001/output.csv'
output_df.to_csv(file_path, index=False)

In [209]:
output2 = []

for i in range(len(output_df)):
    prompt = f"For the following study, respond with the category it best fits into from the following options:\n\
    Tree species classification \n\
    Forest cover assessment \n\
    Canopy and biomass estimation \n\
    Habitat degradation and biodiversity monitoring \n\
    Forest fire monitoring and prevention \n\
    Other \n\
    N/A \n\
    Respond with the category ONLY, exactly as written in the prompt. DO NOT INCLUDE NUMBERS, other commentary, or anything else.\n\n\
    {output_df.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output2.append(response.choices[0].message.content) 

In [217]:
names = pd.read_csv('~/Documents/SOCY3001/SOCY3001/rawdata.csv').iloc[:,0]

output_df['Category'] = output2
output_df['Title'] = names
output_df_to_save = output_df[(output_df != "N/A").all(axis=1)]
output_df_to_save.to_csv('~/Documents/SOCY3001/SOCY3001/output.csv', index=False)
output_df_to_save

pd.read_csv('~/Documents/SOCY3001/SOCY3001/output.csv')

,Location,Data,Goal,Coordinates,Category,Title
0,India,Literature review and AI data,Biodiversity conservation and forest resource ...,"20.593684, 78.96288",Habitat degradation and biodiversity monitoring,Potential for Artificial Intelligence (AI) and...
1,Not specified,Unmanned aerial vehicle imagery,Individual tree species classification,Not specified,Tree species classification,Improving Artificial-Intelligence-Based Indivi...
2,Konya,Individual tree measurements,Tree height and diameter prediction,"37.87198, 32.48458",Canopy and biomass estimation,Innovative deep learning artificial intelligen...
3,Sweden,Airborne Laser Scanning data,Estimating forest biophysical parameters,"61.32509, 14.19977",Canopy and biomass estimation,FOREST BIOPHYSICAL PARAMETER ESTIMATION VIA MA...
4,Turkey,Measured data from sample plots,Predicting relationships between tree height a...,"40.18333, 29.06667",Canopy and biomass estimation,Artificial intelligence with deep learning alg...
...,...,...,...,...,...,...
3254,Amazon forest,LiDAR remote sensing,Tree phenology,"3.4653, -62.2159",Other,Seasonal and drought-related changes in leaf a...
3255,Not specified,Not specified,Risk assessment of El Nino impacts on tropical...,Not specified,Other,A Spatial and Temporal Risk Assessment of the ...
3256,Brazil,Not specified,Plant-vertebrate interaction study,"-15.7801, -47.9292",Other,ATLANTIC POLLINATION: a data set of flowers an...
3257,Not specified,Not specified,Insect herbivory study,"51.16569, 10.45153",Habitat degradation and biodiversity monitoring,Herbivory on the pedunculate oak along an urba...
